## Introduction to TensorFlow ## 

(Much of this material is originally from cs224d TensorFlow tutorial by Bharath Ramsundar)

![TensorFlow logo](images/tensorflow.png)

TensorFlow provides primitives for defining functions on tensors and automatically computing their derivatives. 

* TensorFlow is a deep learning library for Python that has been recently open-sourced by Google. 
* TensorFlow has better support for distributed systems than many other competing libraries (i.e. Theano). 
* Keras (next tutorial) is a  high-level library that builds on TensorFlow. 


## What is a tensor? ##

![tensor definition](images/tensor_definition.png) 

## There are some similarities between TensorFlow and Numpy ##

* Both TensorFlow and Numpy are N-d array libraries 
* Numpy does not have methods to create tensor functions and automatically compute derivatives. 
* Numpy does not have GPU support, but TensorFlow does. 

### Numpy: ###


In [1]:
import numpy as np 

In [2]:
a=np.zeros((2,2)); b=np.ones((2,2))

In [3]:
np.sum(b,axis=1)

array([ 2.,  2.])

In [4]:
a.shape

(2, 2)

In [5]:
np.reshape(a,(1,4))

array([[ 0.,  0.,  0.,  0.]])

### Same commands in TensorFlow:###

In [6]:
import tensorflow as tf

In [8]:
tf.InteractiveSession()

We just created an interactive Session. A Session object encapsulates the environment in which tensors are evaluated. 

In [9]:
 a = tf.zeros((2,2)); b = tf.ones((2,2))

In [10]:
 tf.reduce_sum(b, reduction_indices=1).eval()

array([ 2.,  2.], dtype=float32)

In [11]:
 a.get_shape()

TensorShape([Dimension(2), Dimension(2)])

We see above that TensorShape behaves like a Python  tuple. 

In [12]:
 tf.reshape(a, (1, 4)).eval()

array([[ 0.,  0.,  0.,  0.]], dtype=float32)

We can build a Numpy to TensorFlow dictionary: 
![Numpy To TensorFlow dictionary](images/numpy_to_tensorflow.png)

## TensorFlow requires explicit evaluation ##
TensorFlow computations define a computation graph that has no value until evaluated. Specifically TensorFlow programs usually have two phases: 

* construction phase -- assembles the computation graph 
* evaluation phase -- uses a Session to execute operations in the graph ; all computations add nodes to the global default graph. 

In [13]:
#in Numpy: 
a=np.zeros((2,2))
print(a)

[[ 0.  0.]
 [ 0.  0.]]


In [14]:
#but in TensorFlow
ta=tf.zeros((2,2))
print(ta)

Tensor("zeros_1:0", shape=(2, 2), dtype=float32)


In [15]:
#now, we evaluate the computation graph: 
print(ta.eval())

[[ 0.  0.]
 [ 0.  0.]]


## More on Sessions ##

In [16]:
a=tf.constant(5.0)
b=tf.constant(6.0)
c=a*b 
with tf.Session() as sess: 
    print(sess.run(c))
    print(c.eval())


30.0
30.0


So we observe that ```c.eval()``` is a compact way of executing ```sess.run(c)``` in the currently active session.
```tf.InteractiveSession()``` is convenient syntax for keeping a default session open in iPython. 

## Variables ##

Variables are in-memory buffers that contain tensors. They are used to  hold and update parameters when a model is trained. 

In [18]:
W1 = tf.ones((2,2))
W2 = tf.Variable(tf.zeros((2,2)), name="weights")

with tf.Session() as sess:
 print(sess.run(W1))
 sess.run(tf.global_variables_initializer())
 print(sess.run(W2))

[[ 1.  1.]
 [ 1.  1.]]
[[ 0.  0.]
 [ 0.  0.]]


Unlike constant tensors, TensorFlow variables must be initialized before they have values. 

In [20]:
#variable objects can be initialized from either constants or random values: 
W=tf.Variable(tf.zeros((2,2)), name="weights") # initialized from zero values 
R=tf.Variable(tf.random_normal((2,2)), name="random_weights") #initialized from random values 

#initialize all variables with values specified above: 
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    print(sess.run(W))
    print(sess.run(R))
    

[[ 0.  0.]
 [ 0.  0.]]
[[ 0.71444696 -0.07307655]
 [ 0.18373618  1.137761  ]]


Updating variable state:

In [22]:
state = tf.Variable(0, name="counter")

#new_value = state + 1
new_value = tf.add(state, tf.constant(1))

#state=new_value
update = tf.assign(state, new_value)

with tf.Session() as sess:
    #state=0 
    sess.run(tf.global_variables_initializer())
    #print(state)
    print(sess.run(state))
    for _ in range(3):
        #state=state+1
        sess.run(update)
        #print(state)
        print(sess.run(state))

0
1
2
3


Fetching variable state: 

* Calling ```sess.run(var)``` on a ```tf.Session()``` object retrieves its value. 
* We can retrieve multiple variables simultaneously with ```sess.run([var1,var2])```

For example, let's evaluate the following computational graph: 
![Computation Graph Eval Example](images/comp_graph_eval.png) 


In [24]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
prod = tf.multiply(input1, intermed)
with tf.Session() as sess:
    result = sess.run([prod, intermed])
    print(result)

[21.0, 7.0]


Data inputs to TensorFlow: 

In [25]:
#importing data from a numpy array with "convert_to_tensor" function 
a=np.zeros((3,3))
ta=tf.convert_to_tensor(a)
with tf.Session() as sess: 
    print(sess.run(ta))

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


A more scalable approach: 
* use ```tf.placeholder``` variablesl (dummy nodes that provide entry points for data to the computational graph) 
* a ```feed_dict``` is a Python dictionary mapping from ```tf.placeholder``` variables to data 

![placeholders and feed forward dictionaries](images/placeholder_feedforward_dict.png)

In [27]:
#define placeholder objects for data entry 
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)

output = tf.multiply(input1,input2)
with tf.Session() as sess: 
    #fetch value of output from computational graph and 
    #feed data into the computational graph 
    print(sess.run([output], feed_dict={input1:[7.],input2:[2.]}))
    

[array([ 14.], dtype=float32)]


Variable scope is necessary to avoid name clashes between variables in complex models. 
* ```tf.variable_scope()``` provides simple name-spacing 
* ```tf.get_variable()``` creates/accesses variables from within a variable scope 

In [30]:
#setting a variable's scope adds the corresponding prefix to the variable name 
with tf.variable_scope("foo",reuse=None):
    with tf.variable_scope("bar",reuse=None):
        v = tf.get_variable("v", [1])
assert v.name == "foo/bar/v:0"

In [32]:
with tf.variable_scope("foo",reuse=None):
    v = tf.get_variable("v", [1])
    tf.get_variable_scope().reuse_variables()
    v1 = tf.get_variable("v", [1])
assert v1 == v

```get_variable()``` will behave differently depending on whether or not reuse is enabled.

In [33]:
#case 1: reuse is set to false 
# A new variable is created and returned -- but this will give an error if the variable already exists in this scope, 
#as is the case here 

#with tf.variable_scope("foo"): 
#    v=tf.get_variable("v", [1])
#assert v.name=="foo/v:0"

In [34]:
#case 2: reuse is set to true 
# search for existing variable with a given name 
#raise ValueError if none is found 
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
assert v1 == v

TensorFlow supports auto-differentiation to compute gradients without user input.
* ```tf.train.Optimizer``` creates an optimizer. 
* ```tf.train.Optimizer.minimize(loss, var_list)``` adds optimization operation to the computation graph. 

Check out TensorBoard for visualizing the computational graph and training metrics: https://www.tensorflow.org/versions/r0.11/how_tos/summaries_and_tensorboard/index.html


## MNIST ConvNet Example ##

A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
